In [1]:
import os
import glob
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from keras import regularizers, optimizers

def load_data(root, vfold_ratio=0.3, max_items_per_class=1000):
    all_files = glob.glob(os.path.join(root, '*.npy'))

    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []

    for idx, file in enumerate(all_files):
        data = np.load(file)
        data = data[0:max_items_per_class, :]
        labels = np.full(data.shape[0], idx)

        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)

        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)

    data = None
    labels = None

    permutation = np.random.permutation(y.shape[0])
    x = x[permutation, :]
    y = y[permutation]

    vfold_size = int(x.shape[0]/10*(vfold_ratio*10))

    x_test = x[0:vfold_size, :]
    y_test = y[0:vfold_size]

    x_train = x[vfold_size:x.shape[0], :]
    y_train = y[vfold_size:y.shape[0]]

    return x_train, y_train, x_test, y_test, class_names


x_train, y_train, x_test, y_test, class_names = load_data('./Dataset')

num_classes = len(class_names)
image_size = 28
# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], image_size, image_size).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size).astype('float32')

x_train /= 255.0
x_test /= 255.0
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

'''for i in y_train:
    if i[0]==1 or i[3]== 1:
        i[4]=1
    else:
        i[5] = 1  
for i in y_test:
    if i[0]==1 or i[3]== 1:
        i[4]=1
    else:
        i[5] = 1  
'''
print('Training data: ', x_train.shape)
print('Training labels: ', y_train.shape)
print('Test data: ', x_test.shape)
print('Test labels: ', y_test.shape)
print('Class names:', class_names)

batch_size = 10
num_classes = 4
epochs = 50

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
    x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)
    
model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5),
                 activation='tanh',
                 input_shape=input_shape, kernel_initializer= 'VarianceScaling'))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3), activation='tanh',kernel_initializer='VarianceScaling'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), activation='tanh',kernel_initializer='VarianceScaling'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3), activation='tanh',kernel_initializer='VarianceScaling'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
#model.add(Conv2D(512, (5, 5), activation='tanh'))
model.add(Flatten())
model.add(Dense(1024, activation='relu',kernel_initializer='VarianceScaling'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu',kernel_initializer='VarianceScaling'))

model.add(Dense(num_classes, activation='sigmoid'))
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[chk],
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.
W0720 17:27:21.300786 140708191938304 deprecation_wrapper.py:119] From /home/sujit/anaconda3/envs/mlenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0720 17:27:21.326068 140708191938304 deprecation_wrapper.py:119] From /home/sujit/anaconda3/envs/mlenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0720 17:27:21.328860 140708191938304 deprecation_wrapper.py:119] From /home/sujit/anaconda3/envs/mlenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0720 17:27:21.354033 140708191938304 deprecation_wrapper.py:119] From /home/sujit/anaconda3/envs/mlenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name

Training data:  (2800, 28, 28)
Training labels:  (2800, 4)
Test data:  (1200, 28, 28)
Test labels:  (1200, 4)
Class names: ['apple', 'dog', 'cat', 'strawberry']


W0720 17:27:21.619784 140708191938304 deprecation_wrapper.py:119] From /home/sujit/anaconda3/envs/mlenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0720 17:27:21.662326 140708191938304 deprecation_wrapper.py:119] From /home/sujit/anaconda3/envs/mlenv/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0720 17:27:21.680804 140708191938304 deprecation.py:323] From /home/sujit/anaconda3/envs/mlenv/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2800 samples, validate on 1200 samples
Epoch 1/50
2800/2800 [==============================] - 7s 2ms/step - loss: 0.4238 - acc: 0.7926 - val_loss: 0.2962 - val_acc: 0.8569

Epoch 00001: val_acc improved from -inf to 0.85687, saving model to best_model.pkl
Epoch 2/50
2800/2800 [==============================] - 6s 2ms/step - loss: 0.3406 - acc: 0.8321 - val_loss: 0.3010 - val_acc: 0.8537

Epoch 00002: val_acc did not improve from 0.85687
Epoch 3/50
2800/2800 [==============================] - 6s 2ms/step - loss: 0.3269 - acc: 0.8455 - val_loss: 0.2788 - val_acc: 0.8760

Epoch 00003: val_acc improved from 0.85687 to 0.87604, saving model to best_model.pkl
Epoch 4/50
2800/2800 [==============================] - 6s 2ms/step - loss: 0.3065 - acc: 0.8533 - val_loss: 0.2892 - val_acc: 0.8683

Epoch 00004: val_acc did not improve from 0.87604
Epoch 5/50
2800/2800 [==============================] - 6s 2ms/step - loss: 0.2971 - acc: 0.8612 - val_loss: 0.2630 - val_acc: 0.8829

Epoch 00

2800/2800 [==============================] - 5s 2ms/step - loss: 0.2097 - acc: 0.9114 - val_loss: 0.1798 - val_acc: 0.9200

Epoch 00042: val_acc did not improve from 0.92937
Epoch 43/50
2800/2800 [==============================] - 5s 2ms/step - loss: 0.2114 - acc: 0.9122 - val_loss: 0.1765 - val_acc: 0.9279

Epoch 00043: val_acc did not improve from 0.92937
Epoch 44/50
2800/2800 [==============================] - 5s 2ms/step - loss: 0.2019 - acc: 0.9161 - val_loss: 0.1889 - val_acc: 0.9242

Epoch 00044: val_acc did not improve from 0.92937
Epoch 45/50
2800/2800 [==============================] - 5s 2ms/step - loss: 0.1962 - acc: 0.9212 - val_loss: 0.1817 - val_acc: 0.9235

Epoch 00045: val_acc did not improve from 0.92937
Epoch 46/50
2800/2800 [==============================] - 7s 2ms/step - loss: 0.2040 - acc: 0.9191 - val_loss: 0.1836 - val_acc: 0.9250

Epoch 00046: val_acc did not improve from 0.92937
Epoch 47/50
2800/2800 [==============================] - 6s 2ms/step - loss: 0.204